# **Fine-tuning Wav2Vec-base for english-russian recognition**

---



This notebook is made based on [this notebook](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_🤗_Transformers.ipynb ) from official HuggingFace page.

In [ ]:
%%capture
!pip install torchaudio -q
!pip install librosa -q
!pip install jiwer -q
!pip install umap-learn -q

In [ ]:
!pip install -U transformers -q

In [ ]:
!pip install -U datasets -q

In [ ]:
# it is needed because there is a problem in packages relations
!pip uninstall numpy
!pip uninstall numba
!pip install numba

In [ ]:
import re
import torch
import random
import librosa
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import (
    load_dataset, 
    load_metric, 
    Audio, 
    concatenate_datasets
)
from transformers import (
    Wav2Vec2Processor, 
    Wav2Vec2ForCTC,
    Wav2Vec2Processor, 
    Wav2Vec2Tokenizer, 
    Wav2Vec2Config, 
    Wav2Vec2FeatureExtractor, 
    Wav2Vec2CTCTokenizer,
    TrainingArguments
)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Prepare Data, Tokenizer, Feature Extractor

### Create Wav2Vec2CTCTokenizer

#### Download Data

In [ ]:
path = 'path/to/timit'
timit = load_dataset("timit_asr", data_dir=path)

In [ ]:
timit

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [ ]:
random.seed(8)
timit['train'] = timit['train'].select(range(2000))
timit['test'] = timit['test'].select(range(500))

In [ ]:
russian_train = load_dataset('common_voice', 'ru', split='train')
russian_test = load_dataset('common_voice', 'ru', split='test')

The data selection is randomized to create datasets with more different voices.

In [ ]:
random.seed(10)
to_select_test = []
while len(to_select_test) != 500:
    n = random.randint(0, 8007)
    if n not in to_select_test:
        to_select_test.append(n)
russian_test = russian_test.select(to_select_test)
to_select_train = []
while len(to_select_train) != 2000:
    n = random.randint(0, 10000)
    if n not in to_select_train:
        to_select_train.append(n)
russian_train = russian_train.select(to_select_train)

Remove the unnecessary columns

In [ ]:
russian_test = russian_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
russian_test = russian_test.rename_column('sentence', 'text')
russian_train = russian_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
russian_train = russian_train.rename_column('sentence', 'text')
timit = timit.remove_columns(['dialect_region', 'sentence_type', 'speaker_id', 'id'])

#### Data preprocessing

1. Deleting chars which are not related to the task

In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
russian_test = russian_test.map(remove_special_characters)
russian_train = russian_train.map(remove_special_characters)
timit = timit.map(remove_special_characters)

2. Creating vocabulary

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = timit['train'].map(
    extract_all_chars, batched=True, 
    batch_size=-1, 
    keep_in_memory=True, 
    remove_columns=timit['train'].column_names
)
vocab_test = timit['test'].map(
    extract_all_chars, batched=True, 
    batch_size=-1, 
    keep_in_memory=True, 
    remove_columns=timit['test'].column_names
)

3. Change cyrillic text to latin

In [ ]:
cyr_to_lat = {
    'а': 'a', 'б': 'b', 'в':'v', 'г': 'g', 'д': 'd', 'е': 'e', 
    'ё': 'ё', 'ж': 'ž', 'з': 'z', 'и': 'i', 'й': 'j', 'к': 'k', 'л': 'l', 
    'м': 'm', 'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 
    'у': 'u', 'ф': 'f', 'х': 'h', 'ц': 'c', 'ч': 'č', 'ш': 'š', 'щ': 'ś', 
    'ъ': 'ʺ', 'ы': 'y', 'ь': 'ʹ', 'э': 'è', 'ю': 'û', 'я': 'â'
}

In [ ]:
def change_text(batch):
    new_text = ''
    for letter in batch['text']:
        try:
            new_text += cyr_to_lat[letter]
        except:
            new_text += letter
    batch['text'] = new_text
    return batch

In [ ]:
russian_train = russian_train.map(change_text)
russian_test = russian_test.map(change_text)

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]) | set(cyr_to_lat.values()))
vocab_dict = {v: k+2 for k, v in enumerate(vocab_list)}
# adding 'UNK' token for unknown segments
# 'PAD' for padding segments
vocab_dict["[UNK]"] = 1
vocab_dict["[PAD]"] = 0
# replace space token to '|'
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab = {v: k for k, v in vocab_dict.items()}

4. Save dictionary for tokenizer

In [ ]:
import json
path = '/working/path/'
with open(path + 'vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

#### Create Tokenizer

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_file=f"{path}vocab.json", 
    unk_token="[UNK]", 
    pad_token="[PAD]", 
    word_delimiter_token="|"
)

### Create XLSR-Wav2Vec2 Feature Extractor

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, 
    sampling_rate=16000, 
    padding_value=0.0, 
    do_normalize=True, 
    return_attention_mask=False
)

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Preprocess Audio


The Wav2Vec input must have sampling rate at 16000.

In [ ]:
russian_test = russian_test.cast_column("audio", Audio(sampling_rate=16_000))
russian_train = russian_train.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # unbatch input
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    # transform text labels into numerical
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    
    return batch

Apply the data preparation function to all examples.

In [ ]:
timit_train_proc = timit['train'].map(prepare_dataset,  remove_columns=timit.column_names["train"], num_proc=4)
timit_test_proc = timit['test'].map(prepare_dataset, remove_columns=timit.column_names["test"], num_proc=4)
russian_test_proc = russian_test.map(prepare_dataset, remove_columns=russian_test.column_names)
russian_train_proc = russian_train.map(prepare_dataset, remove_columns=russian_train.column_names)

In [ ]:
dataset = timit['train']
rand_int = random.randint(0, len(dataset))

print("Target text:", dataset[rand_int]["text"])
print("Input array shape:", np.asarray(dataset[rand_int]["audio"]["array"]).shape)
print("Sampling rate:", dataset[rand_int]["audio"]["sampling_rate"])

Target text: she had your dark suit in greasy wash water all year 

Input array shape: (51303,)

Sampling rate: 16000


In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(russian_train))

print(russian_train[rand_int]["text"])
ipd.Audio(data=np.asarray(russian_train[rand_int]["audio"]["array"]), autoplay=True, rate=16000)

v naibolʹšej mere kak nam izvestno ot takih konfliktov stradaût imenno ženśiny i devuški 


## Training

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Thus, it is more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator.

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. The predominant metric in ASR is the word error rate (WER).

In [ ]:
wer_metric = load_metric("wer")

/tmp/ipykernel_34/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print(pred_str[0], label_str[0])
    print(pred_str[-1], label_str[-1])
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

We should concatenate English and Russian datasets to prepare for the input.

In [ ]:
test = concatenate_datasets([timit['test'], russian_test])
test_proc = concatenate_datasets([timit_test_proc, russian_test_proc])
train_proc = concatenate_datasets([timit_train_proc, russian_train_proc])

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_ID = 'facebook/wav2vec2-base'

# FINAL
config = Wav2Vec2Config.from_pretrained(
    MODEL_ID, 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ctc_loss_reduction="mean",
    output_hidden_states=True
)
model = Wav2Vec2ForCTC.from_pretrained(
    MODEL_ID, 
    config=config,
    ignore_mismatched_sizes=True
).to(device)

The feature extractor is freezed as it has been sufficiently trained during pre-training.

In [ ]:
model.freeze_feature_extractor()

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
repo_name = "repository/name"
training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  num_train_epochs=100,
  fp16=True,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=100,
  save_strategy='epoch',
  evaluation_strategy='epoch'
)

In [ ]:
trainer = CTCTrainer(
    get_test_states=True,
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_proc,
    eval_dataset=test_proc,
    tokenizer=processor.feature_extractor,
)

### Training

In [ ]:
import gc
gc.collect()

4321

The model checkpoints are in the dolfer at Google Disk: https://drive.google.com/drive/folders/1WUECVAeEXvcyI42kZ8GhBkoPGgmEGGYS?usp=share_link

In [ ]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer
1,No log,3.237621,1.000000
2,4.551400,3.157347,1.000000
3,3.093600,3.098700,1.000000
4,2.242600,1.191277,0.781061
5,2.242600,0.873133,0.699220
6,1.057500,0.845601,0.667253
7,0.765100,0.783346,0.643854
8,0.631600,0.779965,0.633857
9,0.631600,0.726073,0.605954
10,0.527200,0.769074,0.587169


...

she had your dark suit in greasy wash water all year she had your dark suit in greasy wash water all year
analogičny mubrazovnutrnââ èkoomičeskoâ politika kuby sozdaût serʹeznoj pripâtstven na butisobstennu èkonomičeskogo razvitiâ analogičnym obrazom vnutrennââ èkonomičeskaâ politika kuby sozdaet serʹeznye prepâtstviâ na puti sobstvennogo èkonomičeskogo razvitiâ


/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


she had your dark suit in greasy wash water all year she had your dark suit in greasy wash water all year
onalogičny my brazovnutrnâ èkoomičeskoâ politi ka kuby sozdaût serʹeznoâ prepârtstve na puti sobstvnno èkonomičeskogo razitiâ analogičnym obrazom vnutrennââ èkonomičeskaâ politika kuby sozdaet serʹeznye prepâtstviâ na puti sobstvennogo èkonomičeskogo razvitiâ


/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


KeyboardInterrupt: 